In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import openpyxl
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa as smtsa
from statsmodels.tsa.stattools import bds
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt

In [ ]:
# Caminho para o direório
root_dir = "/content/drive/MyDrive"

# Get the absolute path to the file.
file_path = os.path.join(root_dir, "economatica_nyse_diario.xlsx")

workbook = openpyxl.load_workbook(filename=file_path, read_only=True)

worksheet = workbook["PETR4"]  # Access the desired sheet

rows = []
for row in worksheet.iter_rows():  # Iterate over rows of the worksheet
    rows.append([cell.value for cell in row])

df = pd.DataFrame(rows)

# Limpeza dos dados


Adicionando o nome das colunas no data frame

In [ ]:
df_clean = df[3:]
df_clean.columns = df_clean.iloc[0]
df_clean = df_clean[1:]

colunas = df_clean.columns

colunas_novas = ["Data", "Orig"]

for i in range(2, len(colunas)):
    colunas_novas.append(colunas[i][-10:])

colunas_novas


['Data',
 'Orig',
 'a orig\nDDD',
 'a orig\nMMM',
 'a orig\nAOS',
 'a orig\nAKA',
 ' orig\nATEN',
 'a orig\nAIR',
 'a orig\nABT',
 ' orig\nABBV',
 'a orig\nANF',
 'a orig\nABM',
 'a orig\nAKR',
 ' orig\nACCO',
 'a orig\nAYI',
 ' orig\nGOLF',
 'a orig\nADT',
 ' orig\nATGE',
 'a orig\nAAP',
 'a orig\nWMS',
 ' orig\nASIX',
 'a orig\nACM',
 ' orig\nAJRD',
 'a orig\nAES',
 'a orig\nAMG',
 'a orig\nAFL',
 ' orig\nMITT',
 ' orig\nAGCO',
 'eda orig\nA',
 ' orig\nAGTI',
 'a orig\nAGL',
 'a orig\nADC',
 'da orig\nAL',
 'a orig\nAPD',
 'a orig\nALG',
 'a orig\nALK',
 'a orig\nAIN',
 'a orig\nALB',
 'a orig\nACI',
 'da orig\nAA',
 ' orig\nALEX',
 'a orig\nALX',
 'a orig\nARE',
 'orig\nY-old',
 'a orig\nATI',
 'a orig\nALE',
 'a orig\nBFH',
 'a orig\nLNT',
 ' orig\nALSN',
 'a orig\nALL',
 ' orig\nALLY',
 ' orig\nPINE',
 'a orig\nAYX',
 ' orig\nATUS',
 'da orig\nMO',
 ' orig\nAMAM',
 'a orig\nAMC',
 'a orig\nAEE',
 ' orig\nAMRC',
 'a orig\nAAT',
 'a orig\nAXL',
 'ig\nACC-old',
 'a orig\nAEO',
 'a or

Extraindo apenas o código da ação no nome da coluna


In [ ]:
empresas = ["Data", "Orig"]

for col in colunas_novas[2:]:
    if col.startswith("g\n"):
        empresas.append(col[2:])
    elif col.startswith("ig\n"):
        empresas.append(col[3:])
    elif col.startswith("rig\n"):
        empresas.append(col[4:])
    elif col.startswith("orig\n"):
        empresas.append(col[5:])
    elif col.startswith(" orig\n"):
        empresas.append(col[6:])
    elif col.startswith("a orig\n"):
        empresas.append(col[7:])
    elif col.startswith("da orig\n"):
        empresas.append(col[8:])
    elif col.startswith("eda orig\n"):
        empresas.append(col[9:])
    else:
        empresas.append(col)


empresas

['Data',
 'Orig',
 'DDD',
 'MMM',
 'AOS',
 'AKA',
 'ATEN',
 'AIR',
 'ABT',
 'ABBV',
 'ANF',
 'ABM',
 'AKR',
 'ACCO',
 'AYI',
 'GOLF',
 'ADT',
 'ATGE',
 'AAP',
 'WMS',
 'ASIX',
 'ACM',
 'AJRD',
 'AES',
 'AMG',
 'AFL',
 'MITT',
 'AGCO',
 'A',
 'AGTI',
 'AGL',
 'ADC',
 'AL',
 'APD',
 'ALG',
 'ALK',
 'AIN',
 'ALB',
 'ACI',
 'AA',
 'ALEX',
 'ALX',
 'ARE',
 'Y-old',
 'ATI',
 'ALE',
 'BFH',
 'LNT',
 'ALSN',
 'ALL',
 'ALLY',
 'PINE',
 'AYX',
 'ATUS',
 'MO',
 'AMAM',
 'AMC',
 'AEE',
 'AMRC',
 'AAT',
 'AXL',
 'ACC-old',
 'AEO',
 'AEP',
 'AEL',
 'AXP',
 'AFG',
 'AMH',
 'AIG',
 'ARL',
 'AWR',
 'AMT',
 'AVD',
 'AWK',
 'AMWL',
 'COLD',
 'AMP',
 'ABC',
 'AME',
 'AMN',
 'AMRX',
 'AP',
 'APH',
 'AXR',
 'PLAN-old',
 'AOMR',
 'NLY',
 'ANVS',
 'AM',
 'AR',
 'ELV',
 'APA',
 'AIV',
 'ARI',
 'APO',
 'APLE',
 'AIT',
 'ATR',
 'ARMK',
 'ABR',
 'ARCH',
 'ADM',
 'AROC',
 'ARNC',
 'ACA',
 'RCUS',
 'ACRE',
 'ARES',
 'AGX',
 'ARIS',
 'ANET',
 'AAIC',
 'ARLO',
 'AHH',
 'ARR',
 'AFI',
 'AWI',
 'ARW',
 'AJG',
 'APAM',


O próximo chunk de código foi feito pra substituir os "-" por NaN e transformar a coluna "Data" no formato datetime Year-Month-Day.

In [ ]:
df_clean.columns = empresas

df_clean

import numpy as np

for i in empresas[2:]:
    df_clean[i] = pd.to_numeric(df_clean[i], errors='coerce').replace('-', np.nan)

print(df_clean)

df_clean['Data'] = pd.to_datetime(df_clean['Data'], format='%Y-%m-%d', errors='coerce')

print(df_clean["Data"])

                     Data       Orig   DDD     MMM    AOS     AKA   ATEN  \
4     1986-01-02 00:00:00        0.0   NaN     NaN    NaN     NaN    NaN   
5     1986-01-03 00:00:00        0.0   NaN     NaN    NaN     NaN    NaN   
6     1986-01-06 00:00:00        0.0   NaN     NaN    NaN     NaN    NaN   
7     1986-01-07 00:00:00        0.0   NaN     NaN    NaN     NaN    NaN   
8     1986-01-08 00:00:00        0.0   NaN     NaN    NaN     NaN    NaN   
...                   ...        ...   ...     ...    ...     ...    ...   
9739  2023-04-27 00:00:00  23.240438  8.87  105.33  68.56  0.3651  14.14   
9740  2023-04-28 00:00:00       23.7  9.16  106.22  68.29  0.3671  14.14   
9741  2023-05-01 00:00:00          -  9.19  105.85  70.14  0.3453  14.34   
9742  2023-05-02 00:00:00      22.74  8.97  102.98  69.78  0.3264  14.04   
9743  2023-05-03 00:00:00      22.66  8.85  102.83  69.46  0.3230  14.08   

        AIR     ABT    ABBV  ...  YEXT   YUMC     YUM  ZEN-old  ZVIA     ZBH  \
4      

Exclui todos os valores que foram obtidos no fim de semana. Além disso, seleciona um subconjunto do dataframe contendo apenas colunas com mais de 2000 observações não NA

In [ ]:
dados = df_clean[~df_clean['Data'].dt.dayofweek.isin([5, 6])]

empresas = []
for col in dados.columns[1:]:
    ncols = dados[col].dropna().shape[0]
    if (ncols > 2000):
        empresas.append(col)


empresas = ["Data"] + empresas
dados2 = dados[empresas]

Para todos os retornos de ações, verifica se são estacionários ou não

In [ ]:
estacionario ={}
for col in empresas[1:]:
    acao = dados[col].dropna()
    retorno = 100 * np.log(acao / acao.shift(1))
    p_valor = adfuller(retorno.dropna())[1]
    if (p_valor < 0.05):
        estacionario[col] = ["Estacionario", p_valor]
    else:
        estacionario[col] = ["Não estacionario", p_valor]

empresas_est = list(estacionario.keys())

Exporta o conjunto de dados como csv


In [ ]:
dados2.to_csv("/content/drive/MyDrive/dados_nyse_diario_expandido.csv", index = False)


1126
